## Notebook for plotting data atop the catchment area

In [23]:
# Initialisation cell

# For local notebook
# %load_ext autoreload
# %autoreload 2
# %config IPCompleter.greedy=True

# import os
# import sys
# sys.path.append("../") # go to parent dir
# from src.constants import DATA_PATH, PROJECT_PATH

# For Google Colab
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install if necessary
%pip install geopandas folium seaborn

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 10.9MB 16.0MB/s 
     |████████████████████████████████| 14.8MB 302kB/s 


In [29]:
# Import packages
# Data handling
import numpy as np
import pandas as pd
import geopandas as gpd

# Plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
import folium
import folium.plugins
from IPython.display import IFrame

In [27]:
# Local
# asset_file = os.path.join(DATA_PATH, "Shipston Assets 8.10 Cambridge snapshot (first 200 assets).xlsx")

# Google Colab
asset_file = "drive/My Drive/EA_hackathon/data/Shipston Assets 8.10 Cambridge snapshot (first 200 assets).xlsx"

# Import data
nfm_assets = pd.read_excel(asset_file, header=1)
# Rename longitude and latitude coords
nfm_assets = nfm_assets.rename(columns={"x": "longitude", "y": "latitude"})

# Get the log10 price (due to varying price orders of magnitude)
nfm_assets["log10_price"] = np.log10(nfm_assets["Installed Cost (£)"] + 0.01)

# Finding all asset types
asset_types = nfm_assets["Asset Type"].unique()

In [41]:
# Read in catchment data
# boundaryPath = os.path.join(DATA_PATH, "shapefiles/54106.shp")
boundaryPath = "drive/My Drive/EA_hackathon/data/54106.shp"
boundaryShape = gpd.read_file(boundaryPath)
boundaryJson = boundaryShape.to_crs(epsg=4326).to_json()
style_function = lambda x: {'color': '#ff0000','fill': False}
boundaryPlot = folium.features.GeoJson(boundaryJson, style_function)

# Read in Watercourse data
# watercoursePath = os.path.join(DATA_PATH, "shapefiles/WatercourseLink.shp")
# watercoursePath = "drive/My Drive/EA_hackathon/data/WatercourseLink.shp"
# watercourseShape = gpd.read_file(watercoursePath)
# watercourseJson = watercourseShape.to_crs(epsg=4326).to_json()
# watercoursePlot = folium.features.GeoJson(watercourseJson)

In [42]:
# Plot mapped data
shipston_map = folium.Map(location=[52.120, -1.495],
    tiles="https://api.mapbox.com/styles/v1/croydonbrixton/ckg8jcp6c08vb19o089gcrp0b/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiY3JveWRvbmJyaXh0b24iLCJhIjoiY2tmYTJzZDJiMHJ4YjJ5cXNzMjQzYXd5MSJ9.zUToqkuiUUEdWavpHmXpzQ",
    attr="Mapbox Control Room"
)

asset_color = {}
for asset_type, color in zip(asset_types, sns.color_palette().as_hex()):
    asset_color[asset_type] = color

for i, asset in nfm_assets.iterrows():
    folium.Marker(
        location=(asset.latitude, asset.longitude),
        icon=folium.Icon(color="blue", icon_color=asset_color[asset["Asset Type"]]),
        popup=f"ID:{i}: {asset['Asset Type']}",
        tooltip=f"ID:{i}: {asset['Asset Type']}",
    ).add_to(shipston_map)

shipston_map.add_child(boundaryPlot)
#shipston_map.add_child(watercoursePlot)